In [1]:
from pathlib import Path
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

DRAW_FIGURE = False
folder = Path('./1.width-hist')

if not folder.exists():
    folder.mkdir(parents=True)
    print(f"✅ Created directory: {folder}")
else:
    print(f"📁 Directory already exists: {folder}")


df_l8 = pd.read_csv('../2-preprocess/1.north_glow_datemean_width_timeseries.csv')

stationids = df_l8['stationid'].unique()
df_w = pd.DataFrame(columns=['stationid','w50','w_low','w_high','num'])
df_w = df_w.set_index('stationid')
for s in stationids[:]:
    print('\r... processing %d/%d ...'%(np.where(stationids==s)[0][0]+1, len(stationids)), end='')
    df_all = df_l8[df_l8['stationid']==s]
    df = df_all[df_all['width']>=30]
    if len(df)/len(df_all) < 0.95: continue

    arr_w = df['width'].values
    hist, bins = np.histogram(arr_w, bins=20, range=(np.min(arr_w), np.max(arr_w)))
    freq = hist / len(arr_w)
    freq = freq[:-1]
    df_w.loc[s,'w50'] = np.median(arr_w)
    df_w.loc[s,'w_low'] = bins[np.where(freq>0.03)[0][0]]
    df_w.loc[s,'w_high'] = bins[np.where(freq>0.03)[0][-1] + 1]
    df_w.loc[s,'num'] = len(df)

    if not DRAW_FIGURE: continue

    fig, ax1 = plt.subplots(1,1,figsize=(5,4))
    hs = ax1.hist(df['width'], bins=20, facecolor='#CCCCCC', edgecolor='#666666')
    ax1.plot([bins[0],bins[-1]], [0.03*len(df),0.03*len(df)], c='red', alpha=0.3, zorder=1)
    plt.title('%s n=%d'%(s, len(df)), fontsize=14)
    fig.tight_layout()
    plt.savefig(f'{folder}/{s}.png', dpi=200)

df_w = df_w.sort_index()
df_w.to_csv('1.width_range.csv')


📁 Directory already exists: 1.width-hist
... processing 21/21 ...